## First step to clean the dataset:

- Select only English movies
- Select onlu full-length feature (movies of more than one hour long)
- Remove movies for which we don't have the title

In [1]:
import pandas as pd
import requests

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
character_metadata = pd.read_csv('Data/character.metadata.tsv', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'Freebase_movie_ID',
                                                                              'Movie_release_date',
                                                                              'Character_name',
                                                                              'Actor_date_of_birth',
                                                                              'Actor_gender',
                                                                              'Actor_height',
                                                                              'Actor_ethnicity',
                                                                              'Actor_name',
                                                                              'Actor_age_at_movie_release',
                                                                              'Freebase_character/actor_map_ID',
                                                                              'Freebase_character_ID',
                                                                              'Freebase_actor_ID'])

movie_metadata = pd.read_csv('Data/movie.metadata.tsv', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'Freebase_movie_ID',
                                                                              'Movie_name',
                                                                              'Movie_release_date',
                                                                              'Movie_box_office_revenue',
                                                                              'Movie_runtime',
                                                                              'Movie_languages',
                                                                              'Movie_countries',
                                                                              'Movie_genres'])

plot_summaries = pd.read_csv('Data/plot_summaries.txt', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'summary'])

In [4]:
#Copy of movie_metadata dataset
movie_metadata_filtered = movie_metadata.copy(deep= True)

#Filtering of movie_metada, keeping only full-length features movies (more than 60 minutes long), in English and with that have a Title in the database.
movie_metadata_filtered = movie_metadata_filtered[movie_metadata_filtered.Movie_languages.str.contains("English")]
movie_metadata_filtered.dropna(subset=['Movie_name'],inplace=True)
movie_metadata_filtered = movie_metadata_filtered[movie_metadata_filtered.Movie_runtime >= 60]

print(movie_metadata_filtered)

       Wikipedia_movie_ID Freebase_movie_ID  \
0                  975900         /m/03vyhn   
1                 3196793         /m/08yl5d   
3                 9363483        /m/0285_cd   
6                18998739        /m/04jcqvw   
7                10408933        /m/02qc0j7   
...                   ...               ...   
81735            32468537        /m/0crwd9y   
81736            35228177        /m/0j7hxnt   
81737            34980460        /m/0g4pl34   
81738             9971909        /m/02pygw1   
81740            12476867        /m/02w7zz8   

                                              Movie_name Movie_release_date  \
0                                         Ghosts of Mars         2001-08-24   
1      Getting Away with Murder: The JonBenét Ramsey ...         2000-02-16   
3                                       White Of The Eye               1987   
6                              The Sorcerer's Apprentice               2002   
7                               Alexande

In [5]:
movie_metadata_filtered.Movie_name[movie_metadata_filtered.Movie_name.str.contains(" la ")].head()

#There are still some Title that are not in English...

21500                              Murder a la Mod
33420    La pelota vasca. La piel contra la piedra
39681                             Café de la plage
40550                         Les Damnés de la mer
45773               Lumumba, la mort d'un prophète
Name: Movie_name, dtype: object

In [6]:
movie_Englsih_only = movie_metadata_filtered[movie_metadata_filtered.Movie_languages == '{"/m/02h40lc": "English Language"}']

In [7]:
movie_Englsih_only.Movie_name[movie_Englsih_only.Movie_name.str.contains(" la ")].head()

21500    Murder a la Mod
70558      Irma la Douce
Name: Movie_name, dtype: object

In [8]:
movie_Englsih_only.to_csv(path_or_buf = "Data/movies_metadata_english_only.csv", sep=',')

In [4]:
# IMDB files 

ratings_imdb = pd.read_table('Data/ratings_imdb.tsv', sep="\t")
ratings_imdb.head()


movies_imdb= pd.read_table('Data/movies_imdb.tsv', sep="\t")

